<a href="https://colab.research.google.com/github/jcmachicao/jinis2025_machicao/blob/main/JINIS2025__simulador_SistCog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Simulador Cognitivo vs Red Neuronal (Caldero)

In [8]:
import numpy as np
import pandas as pd
import plotly.express as px

# ---------- Parámetros generales (originally from lt66PfpISrVk) ----------
n_steps = 80
np.random.seed(42)

# ---------- Simulación de entradas (originally from lt66PfpISrVk) ----------
temperatura = np.linspace(50, 90, n_steps) + np.random.normal(0, 2, n_steps)
acidez = np.linspace(4, 8, n_steps)[::-1] + np.random.normal(0, 0.3, n_steps)
presion = np.random.uniform(1.0, 3.0, n_steps)      # bar
flujo = np.random.uniform(0.5, 1.5, n_steps)        # L/s
nivel = np.random.uniform(0.3, 1.0, n_steps)        # fracción

# ---------- Funciones base (originally from f5w0EdHQSvh6) ----------
def sigmoid(x): return 1 / (1 + np.exp(-x))

# NN simulada: salida continua con ruido leve
def apertura_nn(temp, acid, pres, flow, lvl):
    base = sigmoid(0.06*(temp - 70)) * sigmoid(0.8*(6 - acid))
    influence = 0.2*pres + 0.1*flow + 0.15*lvl
    salida = base + 0.05*influence
    salida += np.random.normal(0, 0.03)  # ruido
    return np.clip(salida, 0, 1)

# Sistema Cognitivo: salida NN * factor decisional
def apertura_cog(temp, acid, pres, flow, lvl):
    base = apertura_nn(temp, acid, pres, flow, lvl)
    factor_decisional = np.random.uniform(0.8, 1.2)
    salida = np.clip(base * factor_decisional, 0, 1)
    return salida, factor_decisional

# ---------- Simulación temporal (originally from SCj5q3DOHC2o, defines df) ----------
data = []
for t, (temp, acid, pres, flow, lvl) in enumerate(zip(temperatura, acidez, presion, flujo, nivel)):
    # NN
    psalida_nn = apertura_nn(temp, acid, pres, flow, lvl)
    data.append(["Neuronal", t, temp, acid, psalida_nn, np.nan])

    # Cognitivo
    psalida_cog, fdec = apertura_cog(temp, acid, pres, flow, lvl)
    data.append(["Cognitivo", t, temp, acid, psalida_cog, fdec])

df = pd.DataFrame(data, columns=["Agente", "Tiempo", "Temperatura", "Acidez", "Apertura", "Factor_decisional"])
df["Apertura"] = df["Apertura"] * 100  # escala para el tamaño visual

# --- Data Preparation for Discrepancy ---
# Filter data for each agent
df_neuronal = df[df['Agente'] == 'Neuronal'].copy()
df_cognitivo = df[df['Agente'] == 'Cognitivo'].copy()

# Merge on common columns to calculate difference
df_merged_aperturas = pd.merge(
    df_neuronal[['Tiempo', 'Temperatura', 'Acidez', 'Apertura']],
    df_cognitivo[['Tiempo', 'Apertura']],
    on='Tiempo',
    suffixes=('_neuronal', '_cognitivo')
)

# Calculate absolute discrepancy
df_merged_aperturas['Discrepancia_val'] = np.abs(df_merged_aperturas['Apertura_cognitivo'] - df_merged_aperturas['Apertura_neuronal'])

# Create new rows for the 'Discrepancia' agent
df_discrepancy_rows = pd.DataFrame({
    'Agente': 'Discrepancia',
    'Tiempo': df_merged_aperturas['Tiempo'],
    'Temperatura': df_merged_aperturas['Temperatura'],
    'Acidez': df_merged_aperturas['Acidez'],
    'Apertura': df_merged_aperturas['Discrepancia_val'],
    'Factor_decisional': np.nan # Discrepancy doesn't have a 'Factor_decisional'
})

# Concatenate with the original DataFrame
df_for_plot = pd.concat([df, df_discrepancy_rows], ignore_index=True)

# --- Visualization ---
# Define explicit color map
color_discrete_map = {
    "Neuronal": "blue", # Default Plotly color for consistency if not specified otherwise
    "Cognitivo": "red",   # Default Plotly color
    "Discrepancia": "black" # Black color for discrepancy
}

# Define explicit symbol map (one square, one circle, and discrepancy as circle)
symbol_map = {
    "Neuronal": "square",
    "Cognitivo": "circle",
    "Discrepancia": "circle" # Circle symbol for discrepancy
}

fig = px.scatter(
    df_for_plot, # Use the extended DataFrame
    x="Temperatura",
    y="Acidez",
    size="Apertura",
    color="Agente",
    symbol="Agente", # Map symbols by Agente column
    animation_frame="Tiempo",
    hover_data={"Factor_decisional": True, "Apertura": True},
    range_x=[45, 95],
    range_y=[3.5, 8.5],
    size_max=40,
    title="🧠 Simulador: NN vs Sistema Cognitivo - Válvula (con Discrepancia)",
    labels={"Temperatura": "Temperatura (°C)", "Acidez": "pH", "Apertura": "Apertura válvula"},
    width=700, height=700,
    color_discrete_map=color_discrete_map, # Apply custom color map
    symbol_map=symbol_map # Apply custom symbol map
)

fig.update_layout(
    plot_bgcolor="white",
    legend_title_text="Tipo de Agente",
    title_x=0.5,
    font=dict(size=14)
)

# Add grid to x and y axes
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')

# Apply 50% opacity uniformly to all traces
fig.update_traces(marker=dict(opacity=0.5, line=dict(width=1, color="DarkSlateGrey")))

fig.show()